# 预加载紫各类资源

In [1]:
# 加入plot支持
%matplotlib inline

# 把当前路径的上级路径作为root
import os
import sys
cur_dir = os.getcwd()
pkg_rootdir = os.path.dirname(cur_dir)
sys.path.append(pkg_rootdir)
os.chdir(pkg_rootdir)
print("当前工作路径：",pkg_rootdir)

from mlstock.data.datasource import DataSource
from mlstock.utils import utils
import logging
import pandas as pd
import numpy as np
import math

from mlstock.ml import train
logger = logging.getLogger(__name__)


当前工作路径： /Users/piginzoo/workspaces/mlstock


Loading BokehJS ...

# 加载数据和特征

In [2]:
utils.init_logger(file=False, log_level=logging.INFO)
start_date = "20180101"
end_date = "20220101"
num = 20
df,names = train.load_data(start_date, end_date, num)

开始初始化日志：file=False, simple=False
日志：创建控制台处理器


2022-08-10 13:44:40,152 - INFO - data_filter.py:37 P12500: 加载[4857]只股票的基础信息(basic)
2022-08-10 13:44:40,155 - INFO - data_filter.py:70 P12500: 过滤掉[0]只不在市股票，剩余[4857]只股票
2022-08-10 13:44:40,167 - INFO - data_filter.py:62 P12500: 过滤掉上市不到[1]年[420]只的股票，剩余[4437]只股票
2022-08-10 13:44:40,172 - INFO - data_filter.py:54 P12500: 过滤掉ST的[146]只股票后，剩余[4291]只股票
2022-08-10 13:44:40,176 - INFO - data_filter.py:23 P12500: 从[4291]只过滤掉非主板、非中小板股票后，剩余[2941]只股票
2022-08-10 13:44:42,409 - INFO - data_loader.py:23 P12500: 加载[20]只股票 20170327~20220101 的周频数据 4783 行，耗时2秒
2022-08-10 13:44:44,520 - INFO - data_loader.py:33 P12500: 加载[20]只股票 20170327~20220101 的日频数据 22635 行，耗时2秒
2022-08-10 13:44:46,785 - INFO - data_loader.py:43 P12500: 加载[20]只股票 20170327~20220101 的日频基础(basic)数据 22635 行，耗时2秒
2022-08-10 13:44:46,862 - INFO - data_loader.py:53 P12500: 加载上证指数 20170327~20220101 的日频数据 1163 行，耗时0秒
2022-08-10 13:44:46,892 - INFO - data_loader.py:62 P12500: 加载上证指数 20170327~20220101 的周频数据 245 行，耗时0秒
2022-08-10 13:44:46,893 - INFO 

# 特征处理

In [33]:
df.describe()
df.trade_date.min(), df.trade_date.max()
df.shape
names
df_features = df[names]
df_data = df[['ts_code','trade_date']+names]


In [32]:
# nan统计，最少的是63.7%的填充度
df_nan_stat = df_features.count() / df_features.shape[0]
df_nan_stat[df_nan_stat < 0.9]

MACD                              0.862011
total_none_current_liabilities    0.891700
cash_in_subtotal_invest           0.891700
cash_in_subtotal_finance          0.848421
EP                                0.872883
alpha                             0.636630
beta                              0.636630
dtype: float64

In [22]:
# inf统计
df_inf = df_features.apply(lambda x:np.isinf(x).sum(), axis=0)
df_inf[df_inf>0]


Series([], dtype: int64)

In [88]:

# 统计特征值都为空的股票（nan sum时候算作0)
df_na_miss_percent_by_code = df_data.groupby(by='ts_code').apply(
    lambda df: (df.shape[0] - df.count()) / df.shape[0])

columns = df_na_miss_percent_by_code.sum()[df_na_miss_percent_by_code.sum()==0].index
df_na_miss_percent_by_code.drop(columns,axis=1)
# [df_na_miss_percent_by_code.sum()>0]
# # 找出最大的那个特征的缺失比，如果其>80%，就剔除这只股票
# df_na_miss_codes = df_na_miss_percent_by_code[df_na_miss_percent_by_code.max(axis=1) > 0.8]['ts_code']
# logger.info("剔除股票%d只，占比%.2f%%", len(df_na_miss_codes), len(df_na_miss_codes)*100/len(df_data.ts_code.unique()))

,return_3w,return_6w,return_12w,turnover_return_3w,turnover_return_6w,turnover_return_12w,std_1w,std_3w,std_6w,std_12w,...,operate_profit_yoy,operate_cashflow_yoy,net_profit_deduct_non_recurring_profit_loss,operate_cashflow_per_share,ROE_TTM,ROA_TTM,ROE_YOY,EP,alpha,beta
ts_code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.008163,0.020408,0.044898,0.012245,0.024490,0.048980,0.016327,0.057143,0.118367,0.240816,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.004082,0.004082
000002.SZ,0.008163,0.020408,0.044898,0.012245,0.024490,0.053061,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.963265,0.963265
000006.SZ,0.009050,0.022624,0.049774,0.013575,0.027149,0.054299,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004525,0.004525
000007.SZ,0.009662,0.024155,0.053140,0.009662,0.024155,0.057971,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.821256,0.884058,0.884058
000008.SZ,0.008475,0.021186,0.046610,0.012712,0.025424,0.050847,0.000000,0.000000,0.000000,0.000000,...,0.004237,0.004237,0.004237,0.004237,0.004237,0.004237,0.004237,0.161017,0.775424,0.775424
000009.SZ,0.008163,0.020408,0.044898,0.012245,0.024490,0.048980,0.000000,0.000000,0.000000,0.000000,...,0.016327,0.016327,0.016327,0.016327,0.016327,0.016327,0.016327,0.000000,0.004082,0.004082
000010.SZ,0.008163,0.020408,0.044898,0.012245,0.024490,0.048980,0.000000,0.000000,0.000000,0.000000,...,0.016327,0.016327,0.016327,0.016327,0.016327,0.016327,0.016327,0.444898,0.004082,0.004082
000011.SZ,0.008163,0.020408,0.044898,0.012245,0.024490,0.048980,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004082,0.004082
000012.SZ,0.008163,0.020408,0.044898,0.012245,0.024490,0.048980,0.000000,0.000000,0.000000,0.000000,...,0.016327,0.016327,0.016327,0.016327,0.016327,0.016327,0.016327,0.000000,0.004082,0.004082


In [71]:
df_features.MACD.isna()

33      False
34      False
35      False
36      False
37      False
        ...  
4778    False
4779    False
4780    False
4781    False
4782    False
Name: MACD, Length: 4123, dtype: bool